In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

In [ ]:
paired_fastq_gz_file_paths = tuple(('../data/{}_1.fq.gz'.format(sample_id),
                                    '../data/{}_2.fq.gz'.format(sample_id))
                                   for sample_id in (
                                       'NC1',
                                       'NC2',
                                       'NC3',
                                       'NM1',
                                       'NM2',
                                       'NM3',
                                       'WC1',
                                       'WC2',
                                       'WC3',
                                       'WM1',
                                       'WM2',
                                       'WM3',
                                   ))

In [ ]:
for fastq_gz_file_paths in paired_fastq_gz_file_paths:

    for fastq_gz_file_path in fastq_gz_file_paths:

        assert os.path.isfile(fastq_gz_file_path), fastq_gz_file_path

In [ ]:
cdna_fasta_gz_file_path = '../../../data/grch/Homo_sapiens.GRCh38.cdna.all.fa.gz'

assert os.path.isfile(cdna_fasta_gz_file_path), cdna_fasta_gz_file_path

In [ ]:
ccal.establish_path('../output/kallisto', 'directory')

tpms = []

for paired_fastq_gz_file_path_0, paired_fastq_gz_file_path_1 in paired_fastq_gz_file_paths:

    sample_id = os.path.commonprefix(
        (paired_fastq_gz_file_path_0.split('/')[-1],
         paired_fastq_gz_file_path_1.split('/')[-1])).strip('_.')

    print(sample_id)

    abundance_file_path = '../output/kallisto/{}/abundance.tsv'.format(
        sample_id)

    if not os.path.isfile(abundance_file_path):

        ccal.count_transcripts_using_kallisto_quant(
            (paired_fastq_gz_file_path_0, paired_fastq_gz_file_path_1),
            cdna_fasta_gz_file_path,
            '../output/kallisto/{}'.format(sample_id),
            n_job=1)

    tpm = pd.read_table(abundance_file_path, index_col=0)['tpm']

    tpm.name = '{}_tpm'.format(sample_id)

    tpms.append(tpm)

    _1d_array = tpm

    _1d_array_without_0 = _1d_array[_1d_array != 0]

    ccal.plot_distributions(
        ('Log(All)', 'Log(Without 0)'),
        (ccal.shift_and_log_nd_array(_1d_array),
         ccal.shift_and_log_nd_array(_1d_array_without_0)),
        plot_rug=False,
        title='{} TPM'.format(sample_id))

enst_x_sample = pd.concat(tpms, axis=1)

enst_x_sample

In [ ]:
enst = pd.read_table('../../../data/enst.tsv', index_col=1)

enst_gene_name = enst['Gene name'].to_dict()

enst

In [ ]:
gene_x_sample = enst_x_sample.copy()

gene_x_sample.index = enst_x_sample.index.map(enst_gene_name.get)

print(gene_x_sample.shape)

gene_x_sample = gene_x_sample.loc[~gene_x_sample.index.isnull()]

print(gene_x_sample.shape)

max_tpm__gene_x_sample = gene_x_sample.groupby(level=0).max()

print(max_tpm__gene_x_sample.shape)

max_tpm__gene_x_sample = max_tpm__gene_x_sample.sort_index()

max_tpm__gene_x_sample.index.name = None

max_tpm__gene_x_sample.columns.name = None

max_tpm__gene_x_sample.to_csv('../output/max_tpm__gene_x_sample.tsv', sep='\t')

max_tpm__gene_x_sample

In [ ]:
max_tpm_log__gene_x_sample = pd.DataFrame(
    ccal.shift_and_log_nd_array(max_tpm__gene_x_sample.values, log_base='2', shift_as_necessary_to_achieve_min=0),
    index=max_tpm__gene_x_sample.index,
    columns=max_tpm__gene_x_sample.columns)

max_tpm_log__gene_x_sample.to_csv(
    '../output/max_tpm_log__gene_x_sample.tsv', sep='\t')

if max_tpm_log__gene_x_sample.size < 1e6:

    ccal.plot_heat_map(
        max_tpm_log__gene_x_sample,
        title='Log(Gene Maximum TPM)',
        xaxis_title='Sample',
        yaxis_title='Gene')

max_tpm_log__gene_x_sample